In [18]:
# sync
import queue

def task(name, work_queue):
    if work_queue.empty():
        print(f"{name} nothing to do")
    else:
        while not work_queue.empty():
            count = work_queue.get()
            print(f'{name} running')
            s = 0
            for i in range(count):
                s += i
            print(f'{name} total: {s}')

def main():
    """
    This is the main entry point for the program.
    """
    work_queue = queue.Queue()
    
    for work in [14, 5, 13, 20]:
        work_queue.put(work)
    
    tasks = [
        (task, 'One', work_queue),
        (task, 'Two', work_queue),
    ]
    
    for t, n, q in tasks:
        t(n, q)


if __name__ == "__main__":
    main()

One running
One total: 91
One running
One total: 10
One running
One total: 78
One running
One total: 190
Two nothing to do


In [40]:
# cooperative parallelism
import queue
import time


def task(name, work_queue):
    while not work_queue.empty():
        delay = work_queue.get()
        print(f'{name} running')
        start = time.time()
        time.sleep(delay)
        print(f'{name} total: {time.time() - start}')
        yield

def main():
    """
    This is the main entry point for the program.
    """
    work_queue = queue.Queue()
    
    for work in [2, 1, 3, 4]:
        work_queue.put(work)
    
    tasks = [
        task('One', work_queue),
        task('Two', work_queue),
    ]
    
    start = time.time()
    done = False
    while not done:
        for t in tasks:
            try:
                next(t)
            except StopIteration:
                tasks.remove(t)
                
            if len(tasks) == 0:
                done = True
    print(f"total tim = {time.time() - start}")



main()

One running
One total: 2.0045557022094727
Two running
Two total: 1.002506971359253
One running
One total: 3.002657175064087
Two running
Two total: 4.003462791442871
total tim = 10.01524829864502


In [44]:
import asyncio
import time

async def task(name, queue):
    while not queue.empty():
        delay = await queue.get()
        start = time.time()
        print(f'{name} running')
        await asyncio.sleep(delay)
        print(f'{name} total: {time.time() - start}')

        
async def main():
    work_queue = asyncio.Queue()
    
    for work in [2, 1, 3, 4]:
        await work_queue.put(work)
    
    tasks = [
        task('One', work_queue),
        task('Two', work_queue),
    ]
    
    start = time.time()
    await asyncio.gather(
        asyncio.create_task(task('One', work_queue)),
        asyncio.create_task(task('Two', work_queue))
    )
    
    print(f"total time = {time.time() - start}")


if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop